In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

path = r"C:\Users\aamir\Desktop\YC\P\ChurnPrediction\src\infrastructure\data\clean_data.csv"
df = spark.read.csv(path, header=True, inferSchema=True)

df.printSchema()
df.show(5)

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)
 |-- gender: double (nullable = true)
 |-- Geography_France: integer (nullable = true)
 |-- Geography_Germany: integer (nullable = true)
 |-- Geography_Spain: integer (nullable = true)

+---------+----------+--------+-----------+---+------+---------+-------------+---------+--------------+---------------+------+------+----------------+-----------------+---------------+
|RowNumber|CustomerId| Surname|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Ex

In [20]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("EncodingExample") \
    .getOrCreate()
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Surname", outputCol="surname_indexed")
df_indexed = indexer.fit(df).transform(df)
# df_indexed.select("Surname", "surname_indexed").show(5)

df = df.drop("Surname")

df_indexed.select("surname_indexed").show(5)




+---------------+
|surname_indexed|
+---------------+
|         1958.0|
|           79.0|
|          336.0|
|          128.0|
|           32.0|
+---------------+
only showing top 5 rows


In [21]:
from pyspark.sql.functions import col

clients_actifs = df.filter(col("Exited") == 0)

clients_inactifs = df.filter(col("Exited") == 1)

print("Clients actifs :", clients_actifs.count())
print("Clients inactifs :", clients_inactifs.count())


Clients actifs : 7963
Clients inactifs : 2037


In [22]:
ratio = clients_inactifs.count() / clients_actifs.count()

clients_actifs_sampled = clients_actifs.sample(withReplacement=False, fraction=ratio, seed=42)

df_balanced = clients_inactifs.union(clients_actifs_sampled)

print("Clients actifs après sous-échantillonnage :", clients_actifs_sampled.count())
print("Clients inactifs :", clients_inactifs.count())


Clients actifs après sous-échantillonnage : 2119
Clients inactifs : 2037


### Sélection des Features

In [23]:
features = []
for c in df_balanced.columns:
    if c != "Exited":
        features.append(c)

print("Features utilisées :", features)

Features utilisées : ['RowNumber', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'gender', 'Geography_France', 'Geography_Germany', 'Geography_Spain']


### Configuration du Préprocessing (VectorAssembler et StandardScaler)

In [24]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

scaler = StandardScaler(inputCol="features_vector", outputCol="features_scaled", withMean=False, withStd=True)


### Configuration du Modèle Random Forest

In [25]:
from pyspark.ml.classification import RandomForestClassifier

rf_model = RandomForestClassifier(labelCol="Exited", featuresCol="features_scaled", numTrees=50, seed=42)


### Construction du Pipeline de Machine Learning

In [26]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, scaler, rf_model])

### Séparation des données : randomSplit([0.8, 0.2], seed=42)

In [27]:
train, test = df_balanced.randomSplit([0.8, 0.2], seed=42)

print("Taille du train :", train.count())
print("Taille du test :", test.count())


Taille du train : 3375
Taille du test : 781


In [28]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col

evaluator = BinaryClassificationEvaluator(
    labelCol="Exited", metricName="areaUnderROC"
)


### Configuration de l'Évaluateur et Validation Croisée

In [29]:
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder() \
    .addGrid(rf_model.numTrees, [50, 100, 150]) \
    .addGrid(rf_model.maxDepth, [5, 10, 15]) \
    .addGrid(rf_model.maxBins, [32, 64]) \
    .addGrid(rf_model.minInstancesPerNode, [1, 2, 5]) \
    .addGrid(rf_model.minInfoGain, [0.0, 0.01, 0.05]) \
    .build()

### Configuration de la Grille de Paramètres (ParamGrid)

In [30]:
cv = CrossValidator(estimator=pipeline,estimatorParamMaps=paramGrid,evaluator=evaluator,numFolds=3)
cv_model = cv.fit(train)


### Entraînement du Modèle avec Validation Croisée

### Prédictions sur les Données de Test

In [31]:
predictions = cv_model.transform(test)
predictions.select("Exited", "prediction", "probability").show(10)

+------+----------+--------------------+
|Exited|prediction|         probability|
+------+----------+--------------------+
|     1|       1.0|[0.37815732004463...|
|     1|       1.0|[0.15762846080269...|
|     1|       1.0|[0.09378367243699...|
|     1|       1.0|[0.18142092102842...|
|     1|       1.0|[0.12181176734999...|
|     1|       0.0|[0.80345577827691...|
|     1|       0.0|[0.75571332927418...|
|     1|       1.0|[0.41325315402413...|
|     1|       1.0|[0.29387924869638...|
|     1|       1.0|[0.49517397784618...|
+------+----------+--------------------+
only showing top 10 rows


### Évaluation des Performances du Modèle

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

auc = evaluator.evaluate(predictions)
print("AUC-ROC :", auc)

accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="Exited", predictionCol="prediction", metricName="accuracy")
accuracy = accuracy_evaluator.evaluate(predictions)
print("Accuracy :", accuracy)

precision_evaluator = MulticlassClassificationEvaluator(labelCol="Exited", predictionCol="prediction", metricName="weightedPrecision")
precision = precision_evaluator.evaluate(predictions)
print("Precision :", precision)

recall_evaluator = MulticlassClassificationEvaluator(labelCol="Exited", predictionCol="prediction", metricName="weightedRecall")
recall = recall_evaluator.evaluate(predictions)
print("Recall :", recall)

f1_evaluator = MulticlassClassificationEvaluator(labelCol="Exited", predictionCol="prediction", metricName="f1")
f1 = f1_evaluator.evaluate(predictions)
print("F1-score :", f1)


AUC-ROC : 0.8701820340324494
Accuracy : 0.7848911651728553
Precision : 0.784753589305652
Recall : 0.7848911651728553
F1-score : 0.7848017127933249


### Sauvegarde du Modèle Entraîné

In [37]:
import os

os.environ["HADOOP_HOME"] = r"search-ms:displayname=Résultats%20de%20la%20recherche%20dans%20hadoop-3.3.1-src&crumb=location:C%3A%5CUsers%5Caamir%5CDownloads%5Chadoop-3.3.1-src%5Chadoop-3.3.1-src"
os.environ["PATH"] += os.pathsep + r"search-ms:displayname=Résultats%20de%20la%20recherche%20dans%20hadoop-3.3.1-src&crumb=location:C%3A%5CUsers%5Caamir%5CDownloads%5Chadoop-3.3.1-src%5Chadoop-3.3.1-src\bin"


In [38]:
model_path = "../src/interfaces/best_pipeline_model"

cv_model.bestModel.write().overwrite().save(model_path)
print(" Model saved successfully")


Py4JJavaError: An error occurred while calling o1238310.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1116)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:798)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:838)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:810)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:837)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdi